In [1]:
import pandas as pd
import numpy as np
from turtle import width
import plotly.graph_objects as go

df = pd.read_excel("bmwConfig-RCK22.xlsx",sheet_name='Slip Target', header=None)

#Pull Off First Column for Mu_x
Target_mu_x = df[0]
Target_mu_x = Target_mu_x.drop(index=df.index[0])

#Pull Off First Row for Lean
targetLean = df.iloc[0]
targetLean = targetLean.drop(index=df.index[0])


#Create 2D Array of Slip Target Values
for x in range(1, len(targetLean)+1):
    yArray=[]
    for y in range(1,len(Target_mu_x)+1):
        yArray.append(df.iloc[x,y])
        if len(yArray) == 10:
            if x == 1:
                targetSlip = np.array([yArray])
            else:
                targetSlip = np.append(targetSlip,[yArray], axis=0)


#Read Log file
log = pd.read_csv("log1.csv")

#Remove second row that has value types and convert whole dataframe to number
log.drop(log.index[0], inplace=True)
log = log.apply(pd.to_numeric)

#Remove Negative Slip (braking)
brakingValues = log[log['slip']<0].index
log.drop(brakingValues, inplace=True)

#Convert All Lean Angles to positive
log['phi_lean'] = log['phi_lean'].abs()

mu_x = log['mu_x_rear']
slip = log['slip']
lean = log['phi_lean']

#Create Mesh Grid from Target Values
targetLean, Target_mu_x = np.meshgrid(targetLean, Target_mu_x)

#Real Values Read from Log File
actualLean=lean
actualmu_x=mu_x
actualSlip=slip

fig = go.Figure(data=[
    go.Surface(x=Target_mu_x, y=targetLean, z=targetSlip, opacity=1, colorscale='rdbu', name='Target Slip'),
    go.Scatter3d(x=actualmu_x, y=actualLean, z=actualSlip,
    name='Actual Slip Turn 6',
    mode='lines',
    line=dict(
        color='green',
        width=8
    )
    )
    ])
fig.update_layout(title='Target Slip vs Actual Slip - Turn 6 Schleiz', autosize=False,
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.update_layout(scene = dict(
                    xaxis_title='mu_x',
                    yaxis_title='Lean Angle (degrees)',
                    zaxis_title='Slip (%)'))

#Needed to make the layout match what you see in the BMW Software
fig.update_layout(
    scene={
        'xaxis': {'autorange': 'reversed'}, # reverse automatically
    }
)


fig.show()
